In [ ]:
! pip install transformers==4.24.0
! pip install datasets 
! pip install sacrebleu

In [2]:
import numpy as np
import torch
import torch.nn as nn
# import language_tool_python
from argparse import Namespace
from datasets import load_dataset, load_metric
from transformers import (
    AutoConfig, 
    AutoTokenizer, 
    AutoModelForSeq2SeqLM,
    AutoModelForCausalLM,
    AutoModelForMaskedLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    default_data_collator,
)

import os
import time
from copy import deepcopy
from tqdm import tqdm
import pandas as pd
from typing import *
import inspect

from transformers.generation_utils import BeamSearchScorer
from transformers import LogitsProcessorList, StoppingCriteriaList

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Blended_Skill_Talk Dataset

In [12]:
bst_dataset = load_dataset("blended_skill_talk")
train_dataset = bst_dataset['train']
eval_dataset = bst_dataset['validation']
test_dataset = bst_dataset['test']
print(bst_dataset)
# print(train_dataset[0])

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['personas', 'additional_context', 'previous_utterance', 'context', 'free_messages', 'guided_messages', 'suggestions', 'guided_chosen_suggestions', 'label_candidates'],
        num_rows: 4819
    })
    validation: Dataset({
        features: ['personas', 'additional_context', 'previous_utterance', 'context', 'free_messages', 'guided_messages', 'suggestions', 'guided_chosen_suggestions', 'label_candidates'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['personas', 'additional_context', 'previous_utterance', 'context', 'free_messages', 'guided_messages', 'suggestions', 'guided_chosen_suggestions', 'label_candidates'],
        num_rows: 980
    })
})


In [9]:
# Get statistics of pair of dialogues 
train_num, eval_num, test_num = 0, 0, 0
for i, instance in enumerate(train_dataset):
    train_num += len(instance['free_messages'])

for i, instance in enumerate(eval_dataset):
    eval_num += len(instance['free_messages'])

for i, instance in enumerate(test_dataset):
    test_num += len(instance['free_messages'])

print("#pairs of training dialogues: {}, validation dialogues: {}, test dialogues: {}".format(
    train_num, eval_num, test_num,
))

#pairs of training dialogues: 27018, validation dialogues: 5651, test dialogues: 5482


In [10]:
# Show examples
for i, instance in enumerate(test_dataset.select(range(1))):
    for (x, y) in zip(instance['free_messages'], instance['guided_messages']):
        print("U--{}\nG--{}".format(x, y))
        print("")

U--and why is that?
G--I think it's because in my head, I think everyone is judging me. I just start to sweat and I get sick in my stomach.

U--interesting but I know how you feel especially the whole people telling that it in your head 
G--I don't really have people telling me in my head, more like behind my back

U--Dang that's though. But I also understand that. I have people some who talks behind my back because of certain things that I believe in 
G--Me too! What do you believe in? I believe in dragons... Just finished watching Game of Thrones. Man, those things are dope



In [13]:
data_args = Namespace(
    # model_name_or_path="facebook/bart-base",
    model_name_or_path="/content/drive/My Drive/results/",
    max_source_length=256,
    max_target_length=256,
    pad_to_max_length=False,
    ignore_pad_token_for_loss=True,
    max_train_samples=None,
    preprocessing_num_workers=None,
    overwrite_cache=True,
    output_dir='.',
)

max_target_length = data_args.max_target_length
padding = "max_length" if data_args.pad_to_max_length else False

In [14]:
config = AutoConfig.from_pretrained(data_args.model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(data_args.model_name_or_path)
model = AutoModelForSeq2SeqLM.from_pretrained(data_args.model_name_or_path)

In [15]:
model

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05,

### Tokenize dataset

In [ ]:
def preprocess_bst(examples):
    inputs = [sent for ex in examples["free_messages"] for sent in ex]
    targets = [sent for ex in examples["guided_messages"] for sent in ex]
    model_inputs = tokenizer(inputs, max_length=data_args.max_source_length, padding=padding, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length" and data_args.ignore_pad_token_for_loss:
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
column_names = train_dataset.column_names

if data_args.max_train_samples is not None:
    train_dataset = train_dataset.select(range(data_args.max_train_samples))

train_dataset = train_dataset.map(
    preprocess_bst,
    batched=True,
    num_proc=data_args.preprocessing_num_workers,
    remove_columns=column_names,
    load_from_cache_file=not data_args.overwrite_cache,
)

eval_dataset = eval_dataset.map(
    preprocess_bst,
    batched=True,
    num_proc=data_args.preprocessing_num_workers,
    remove_columns=column_names,
    load_from_cache_file=not data_args.overwrite_cache,
)

### Training

In [ ]:
# Data collator
label_pad_token_id = -100 if data_args.ignore_pad_token_for_loss else tokenizer.pad_token_id
if data_args.pad_to_max_length:
    data_collator = default_data_collator
else:
    data_collator = DataCollatorForSeq2Seq(
        tokenizer,
        model=model,
        label_pad_token_id=label_pad_token_id,
        pad_to_multiple_of=8 if training_args.fp16 else None,
    )

# Metric
metric = load_metric("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    if data_args.ignore_pad_token_for_loss:
        # Replace -100 in the labels as we can't decode them.
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

# Initialize our Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset if training_args.do_train else None,
    eval_dataset=eval_dataset if training_args.do_eval else None,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics if training_args.predict_with_generate else None,
)

In [ ]:
# Training
checkpoint = None

train_result = trainer.train(resume_from_checkpoint=checkpoint)
trainer.save_model()  # Saves the tokenizer too for easy upload

metrics = train_result.metrics
max_train_samples = (
    data_args.max_train_samples if data_args.max_train_samples is not None else len(train_dataset)
)
metrics["train_samples"] = min(max_train_samples, len(train_dataset))

trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

### Dialogue API

In [16]:
def dialogue(
        module,
        inputs: Optional[torch.Tensor] = None,
        max_length: Optional[int] = None,
        min_length: Optional[int] = None,
        do_sample: Optional[bool] = None,
        early_stopping: Optional[bool] = None,
        num_beams: Optional[int] = None,
        temperature: Optional[float] = None,
        top_k: Optional[int] = None,
        top_p: Optional[float] = None,
        repetition_penalty: Optional[float] = None,
        bad_words_ids: Optional[Iterable[int]] = None,
        bos_token_id: Optional[int] = None,
        pad_token_id: Optional[int] = None,
        eos_token_id: Optional[int] = None,
        length_penalty: Optional[float] = None,
        no_repeat_ngram_size: Optional[int] = None,
        encoder_no_repeat_ngram_size: Optional[int] = None,
        num_return_sequences: Optional[int] = None,
        max_time: Optional[float] = None,
        max_new_tokens: Optional[int] = None,
        decoder_start_token_id: Optional[int] = None,
        use_cache: Optional[bool] = None,
        num_beam_groups: Optional[int] = None,
        diversity_penalty: Optional[float] = None,
        prefix_allowed_tokens_fn: Optional[Callable[[int, torch.Tensor], List[int]]] = None,
        logits_processor: Optional[LogitsProcessorList] = LogitsProcessorList(),
        stopping_criteria: Optional[StoppingCriteriaList] = StoppingCriteriaList(),
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        output_scores: Optional[bool] = None,
        return_dict_in_generate: Optional[bool] = None,
        forced_bos_token_id: Optional[int] = None,
        forced_eos_token_id: Optional[int] = None,
        remove_invalid_values: Optional[bool] = None,
        synced_gpus: Optional[bool] = None,
        **model_kwargs):
    # 1. Set generation parameters if not already defined
    bos_token_id = bos_token_id if bos_token_id is not None else module.config.bos_token_id
    num_beams = num_beams if num_beams is not None else module.config.num_beams
    length_penalty = length_penalty if length_penalty is not None else module.config.length_penalty
    early_stopping = early_stopping if early_stopping is not None else module.config.early_stopping
    num_beam_groups = num_beam_groups if num_beam_groups is not None else module.config.num_beam_groups
    do_sample = do_sample if do_sample is not None else module.config.do_sample
    num_return_sequences = (
        num_return_sequences if num_return_sequences is not None else module.config.num_return_sequences
    )

    pad_token_id = pad_token_id if pad_token_id is not None else module.config.pad_token_id
    eos_token_id = eos_token_id if eos_token_id is not None else module.config.eos_token_id

    # output_scores = output_scores if output_scores is not None else module.config.output_scores
    output_scores = True
    output_attentions = output_attentions if output_attentions is not None else module.config.output_attentions
    output_hidden_states = (
        output_hidden_states if output_hidden_states is not None else module.config.output_hidden_states
    )
    # return_dict_in_generate = (
    #     return_dict_in_generate if return_dict_in_generate is not None else module.config.return_dict_in_generate
    # )
    return_dict_in_generate = True

    if pad_token_id is None and eos_token_id is not None:
        # special case if pad_token_id is not defined
        # logger.warning(f"Setting `pad_token_id` to `eos_token_id`:{eos_token_id} for open-end generation.")
        pad_token_id = eos_token_id

    # 2. Define model inputs
    # inputs_tensor has to be defined
    # model_input_name is defined if model-specific keyword input is passed
    # otherwise model_input_name is None
    # all model-specific keyword inputs are removed from `model_kwargs`
    inputs_tensor, model_input_name, model_kwargs = module._prepare_model_inputs(inputs, bos_token_id, model_kwargs)
    batch_size = inputs_tensor.shape[0]

    # 3. Define other model kwargs
    model_kwargs["output_attentions"] = output_attentions
    model_kwargs["output_hidden_states"] = output_hidden_states
    model_kwargs["use_cache"] = use_cache

    accepts_attention_mask = "attention_mask" in set(inspect.signature(module.forward).parameters.keys())
    requires_attention_mask = "encoder_outputs" not in model_kwargs

    if model_kwargs.get("attention_mask", None) is None and requires_attention_mask and accepts_attention_mask:
        model_kwargs["attention_mask"] = module._prepare_attention_mask_for_generation(
            inputs_tensor, pad_token_id, eos_token_id
        )

    if module.config.is_encoder_decoder and "encoder_outputs" not in model_kwargs:
        # if model is encoder decoder encoder_outputs are created
        # and added to `model_kwargs`
        model_kwargs = module._prepare_encoder_decoder_kwargs_for_generation(
            inputs_tensor, model_kwargs, model_input_name
        )

    # 4. Prepare `input_ids` which will be used for auto-regressive generation
    if module.config.is_encoder_decoder:
        input_ids = module._prepare_decoder_input_ids_for_generation(
            batch_size,
            decoder_start_token_id=decoder_start_token_id,
            bos_token_id=bos_token_id,
            model_kwargs=model_kwargs,
        )
    else:
        # if decoder-only then inputs_tensor has to be `input_ids`
        input_ids = inputs_tensor

    # 5. Prepare `max_length` depending on other stopping criteria
    # if `max_new_tokens` is passed, but not `max_length` -> set `max_length = max_new_tokens`
    if max_length is None and max_new_tokens is not None:
        max_length = max_new_tokens + input_ids.shape[-1]
    elif max_length is not None and max_new_tokens is not None:
        pass
        # Both are set, this is odd, raise a warning
        # warnings.warn(
        #     "Both `max_length` and `max_new_tokens` have been set "
        #     f"but they serve the same purpose. `max_length` {max_length} "
        #     f"will take priority over `max_new_tokens` {max_new_tokens}.",
        #     UserWarning,
        # )
    # default to config if still None
    max_length = max_length if max_length is not None else module.config.max_length

    if input_ids.shape[-1] >= max_length:
        input_ids_string = "decoder_input_ids" if module.config.is_encoder_decoder else "input_ids"
        # logger.warning(
        #     f"Input length of {input_ids_string} is {input_ids.shape[-1]}, but ``max_length`` is set to {max_length}. "
        #     "This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``."
        # )

    # 6. determine generation mode
    is_greedy_gen_mode = (num_beams == 1) and (num_beam_groups == 1) and do_sample is False
    is_sample_gen_mode = (num_beams == 1) and (num_beam_groups == 1) and do_sample is True
    is_beam_gen_mode = (num_beams > 1) and (num_beam_groups == 1) and do_sample is False
    is_beam_sample_gen_mode = (num_beams > 1) and (num_beam_groups == 1) and do_sample is True
    is_group_beam_gen_mode = (num_beams > 1) and (num_beam_groups > 1)

    if num_beam_groups > num_beams:
        raise ValueError("`num_beam_groups` has to be smaller or equal to `num_beams`")
    if is_group_beam_gen_mode and do_sample is True:
        raise ValueError(
            "Diverse beam search cannot be used in sampling mode. Make sure that `do_sample` is set to `False`."
        )

    # # 7. prepare distribution pre_processing samplers
    # logits_processor = module._get_logits_processor(
    #     repetition_penalty=repetition_penalty,
    #     no_repeat_ngram_size=no_repeat_ngram_size,
    #     encoder_no_repeat_ngram_size=encoder_no_repeat_ngram_size,
    #     encoder_input_ids=inputs_tensor,
    #     bad_words_ids=bad_words_ids,
    #     min_length=min_length,
    #     max_length=max_length,
    #     eos_token_id=eos_token_id,
    #     forced_bos_token_id=forced_bos_token_id,
    #     forced_eos_token_id=forced_eos_token_id,
    #     prefix_allowed_tokens_fn=prefix_allowed_tokens_fn,
    #     num_beams=num_beams,
    #     num_beam_groups=num_beam_groups,
    #     diversity_penalty=diversity_penalty,
    #     remove_invalid_values=remove_invalid_values,
    #     logits_processor=logits_processor,
    # )

    # 8. prepare stopping criteria
    stopping_criteria = module._get_stopping_criteria(
        max_length=max_length, max_time=max_time, stopping_criteria=stopping_criteria
    )

    # 9. go into different generation modes
    if is_greedy_gen_mode:
        if num_return_sequences > 1:
            raise ValueError(
                f"num_return_sequences has to be 1, but is {num_return_sequences} when doing greedy search."
            )

        # 10. run greedy search
        return module.greedy_search(
            input_ids,
            logits_processor=logits_processor,
            stopping_criteria=stopping_criteria,
            pad_token_id=pad_token_id,
            eos_token_id=eos_token_id,
            output_scores=output_scores,
            return_dict_in_generate=return_dict_in_generate,
            synced_gpus=synced_gpus,
            **model_kwargs,
        )

    elif is_sample_gen_mode:
        # 10. prepare logits warper
        logits_warper = module._get_logits_warper(
            top_k=top_k, top_p=top_p, temperature=temperature, num_beams=num_beams
        )

        # 11. expand input_ids with `num_return_sequences` additional sequences per batch
        input_ids, model_kwargs = module._expand_inputs_for_generation(
            input_ids,
            expand_size=num_return_sequences,
            is_encoder_decoder=module.config.is_encoder_decoder,
            **model_kwargs,
        )

        # 12. run sample
        return module.sample(
            input_ids,
            logits_processor=logits_processor,
            logits_warper=logits_warper,
            stopping_criteria=stopping_criteria,
            pad_token_id=pad_token_id,
            eos_token_id=eos_token_id,
            output_scores=output_scores,
            return_dict_in_generate=return_dict_in_generate,
            synced_gpus=synced_gpus,
            **model_kwargs,
        )

    elif is_beam_gen_mode:
        if num_return_sequences > num_beams:
            raise ValueError("`num_return_sequences` has to be smaller or equal to `num_beams`.")

        if stopping_criteria.max_length is None:
            raise ValueError("`max_length` needs to be a stopping_criteria for now.")

        # 10. prepare beam search scorer
        beam_scorer = BeamSearchScorer(
            batch_size=batch_size,
            num_beams=num_beams,
            device=module.device,
            length_penalty=length_penalty,
            do_early_stopping=early_stopping,
            num_beam_hyps_to_keep=num_return_sequences,
        )
        # 11. interleave input_ids with `num_beams` additional sequences per batch
        input_ids, model_kwargs = module._expand_inputs_for_generation(
            input_ids, expand_size=num_beams, is_encoder_decoder=module.config.is_encoder_decoder, **model_kwargs
        )
        # 12. run beam search
        return module.beam_search(
            input_ids,
            beam_scorer,
            logits_processor=logits_processor,
            stopping_criteria=stopping_criteria,
            pad_token_id=pad_token_id,
            eos_token_id=eos_token_id,
            output_scores=output_scores,
            return_dict_in_generate=return_dict_in_generate,
            synced_gpus=synced_gpus,
            **model_kwargs,
        )

    elif is_beam_sample_gen_mode:
        # 10. prepare logits warper
        logits_warper = module._get_logits_warper(
            top_k=top_k, top_p=top_p, temperature=temperature, num_beams=num_beams
        )

        if stopping_criteria.max_length is None:
            raise ValueError("`max_length` needs to be a stopping_criteria for now.")
        # 11. prepare beam search scorer
        beam_scorer = BeamSearchScorer(
            batch_size=batch_size * num_return_sequences,
            num_beams=num_beams,
            device=module.device,
            length_penalty=length_penalty,
            do_early_stopping=early_stopping,
        )

        # 12. interleave input_ids with `num_beams` additional sequences per batch
        input_ids, model_kwargs = module._expand_inputs_for_generation(
            input_ids,
            expand_size=num_beams * num_return_sequences,
            is_encoder_decoder=module.config.is_encoder_decoder,
            **model_kwargs,
        )

        # 13. run beam sample
        return module.beam_sample(
            input_ids,
            beam_scorer,
            logits_processor=logits_processor,
            logits_warper=logits_warper,
            stopping_criteria=stopping_criteria,
            pad_token_id=pad_token_id,
            eos_token_id=eos_token_id,
            output_scores=output_scores,
            return_dict_in_generate=return_dict_in_generate,
            synced_gpus=synced_gpus,
            **model_kwargs,
        )

    elif is_group_beam_gen_mode:
        if num_return_sequences > num_beams:
            raise ValueError("`num_return_sequences` has to be smaller or equal to `num_beams`.")

        if num_beams % num_beam_groups != 0:
            raise ValueError("`num_beams` should be divisible by `num_beam_groups` for group beam search.")

        if stopping_criteria.max_length is None:
            raise ValueError("`max_length` needs to be a stopping_criteria for now.")

        # 10. prepare beam search scorer
        beam_scorer = BeamSearchScorer(
            batch_size=batch_size,
            num_beams=num_beams,
            max_length=stopping_criteria.max_length,
            device=module.device,
            length_penalty=length_penalty,
            do_early_stopping=early_stopping,
            num_beam_hyps_to_keep=num_return_sequences,
            num_beam_groups=num_beam_groups,
        )
        # 11. interleave input_ids with `num_beams` additional sequences per batch
        input_ids, model_kwargs = module._expand_inputs_for_generation(
            input_ids, expand_size=num_beams, is_encoder_decoder=module.config.is_encoder_decoder, **model_kwargs
        )
        # 12. run beam search
        return module.group_beam_search(
            input_ids,
            beam_scorer,
            logits_processor=logits_processor,
            stopping_criteria=stopping_criteria,
            pad_token_id=pad_token_id,
            eos_token_id=eos_token_id,
            output_scores=output_scores,
            return_dict_in_generate=return_dict_in_generate,
            synced_gpus=synced_gpus,
            **model_kwargs,
        )

### Attack methods

In [17]:
class BaseAttacker:
    def __init__(self, 
                 device,
                 tokenizer,
                 model,
                 max_len=64,
                 max_per=3):
      
        self.device = device
        self.model = model.to(self.device)
        self.tokenizer = tokenizer

        self.embedding = self.model.get_input_embeddings().weight
        self.specical_token = self.tokenizer.all_special_tokens
        self.specical_id = self.tokenizer.all_special_ids
        self.eos_token_id = self.model.config.eos_token_id
        self.pad_token_id = self.model.config.pad_token_id
        self.num_beams = self.model.config.num_beams
        self.num_beam_groups = self.model.config.num_beam_groups
        self.max_len = max_len
        self.max_per = max_per

        self.softmax = nn.Softmax(dim=1)
        self.bce_loss = nn.BCELoss()

    @classmethod
    def _get_hparam(cls, namespace: Namespace, key: str, default=None):
        if hasattr(namespace, key):
            return getattr(namespace, key)
        print('Using default argument for "{}"'.format(key))

        return default

    def run_attack(self, x):
        pass

    def compute_loss(self, x):
        pass

    def compute_seq_len(self, seq):
        if seq[0].eq(self.pad_token_id):
            return int(len(seq) - sum(seq.eq(self.pad_token_id)))
        else:
            return int(len(seq) - sum(seq.eq(self.pad_token_id))) - 1

    def get_prediction(self, sentence):
        def remove_pad(s):
            for i, tk in enumerate(s):
                if tk == self.eos_token_id and i != 0:
                    return s[:i + 1]
            return s

        input_ids = self.tokenizer(sentence, return_tensors="pt").input_ids.to(self.device)
        
        # ['sequences', 'sequences_scores', 'scores', 'beam_indices'] if num_beams != 1
        # ['sequences', 'scores'] if num_beams = 1
        outputs = dialogue(
            self.model, 
            input_ids,
            early_stopping=False, 
            num_beams=self.num_beams,
            num_beam_groups=self.num_beam_groups, 
            use_cache=True,
            max_length=self.max_len,
        )
        
        seqs = outputs['sequences']
        seqs = [remove_pad(seq) for seq in seqs]
        out_scores = outputs['scores']
        pred_len = [self.compute_seq_len(seq) for seq in seqs]
        return pred_len, seqs, out_scores

    def get_trans_string_len(self, text):
        pred_len, seqs, _ = self.get_prediction(text)
        return seqs[0], pred_len[0]

    def get_trans_len(self, text):
        pred_len, _, _ = self.get_prediction(text)
        return pred_len

    def get_trans_strings(self, text):
        pred_len, seqs, _ = self.get_prediction(text)
        out_res = [self.tokenizer.decode(seq, skip_special_tokens=True) for seq in seqs]
        return out_res, pred_len
    
    def compute_score(self, text):
        batch_size = len(text)
        index_list = [i * self.num_beams for i in range(batch_size + 1)]
        pred_len, seqs, out_scores = self.get_prediction(text)


        scores = [[] for _ in range(batch_size)]
        for out_s in out_scores:
            for i in range(batch_size):
                current_index = index_list[i]
                scores[i].append(out_s[current_index: current_index + 1])
        scores = [torch.cat(s) for s in scores]
        scores = [s[:pred_len[i]] for i, s in enumerate(scores)]
        return scores, seqs, pred_len

In [18]:
class SlowAttacker(BaseAttacker):
    def __init__(self, 
                 device,
                 tokenizer,
                 model,
                 max_len=64,
                 max_per=3):
        super(SlowAttacker, self).__init__(device, tokenizer, model, max_len, max_per)

    def leave_eos_loss(self, scores, pred_len):
        loss = []
        for i, s in enumerate(scores):
            s[:, self.pad_token_id] = 1e-12 # T X V
            eos_p = self.softmax(s)[:pred_len[i], self.eos_token_id]
            loss.append(self.bce_loss(eos_p, torch.zeros_like(eos_p)))
        return loss

    def leave_eos_target_loss(self, scores, seqs, pred_len):
        loss = []
        for i, s in enumerate(scores): # s: T X V
            # if self.pad_token_id != self.eos_token_id:
            s[:, self.pad_token_id] = 1e-12
            softmax_v = self.softmax(s)
            eos_p = softmax_v[:pred_len[i], self.eos_token_id]
            target_p = torch.stack([softmax_v[idx, s] for idx, s in enumerate(seqs[i][1:])])
            target_p = target_p[:pred_len[i]]
            pred = eos_p + target_p
            pred[-1] = pred[-1] / 2
            loss.append(self.bce_loss(pred, torch.zeros_like(pred)))
        return loss

    @torch.no_grad()
    def select_best(self, new_strings, batch_size=30):
        """
        Select generated strings which induce longest output sentences.
        """
        pred_len = []
        # seqs = []
        batch_num = len(new_strings) // batch_size
        if batch_size * batch_num != len(new_strings):
            batch_num += 1

        for i in range(batch_num):
            st, ed = i * batch_size, min(i * batch_size + batch_size, len(new_strings))
            input_ids = self.tokenizer(new_strings[st:ed], return_tensors="pt", padding=True).input_ids
            input_ids = input_ids.to(self.device)
            outputs = self.model.generate(
                input_ids, 
                num_beams=self.num_beams, 
                max_length=self.max_len,
                return_dict_in_generate=True,
            )
            lengths = [self.compute_seq_len(seq) for seq in outputs['sequences']]
            # pdb.set_trace()
            pred_len.extend(lengths)
            
        # pred_len = np.array([self.compute_seq_len(torch.tensor(seq)) for seq in seqs])
        pred_len = np.array(pred_len)
        # pdb.set_trace()

        assert len(new_strings) == len(pred_len)
        return new_strings[pred_len.argmax()], max(pred_len)

    def prepare_attack(self, text):
        ori_len = self.get_trans_len(text)[0] # original sentence length
        best_adv_text, best_len = deepcopy(text), ori_len
        current_adv_text, current_len = deepcopy(text), ori_len  # current_adv_text: List[str]
        return ori_len, (best_adv_text, best_len), (current_adv_text, current_len)

    def compute_loss(self, text):
        raise NotImplementedError

    def mutation(self, current_adv_text, grad, modified_pos):
        raise NotImplementedError

    def run_attack(self, text):
        """
        (1) Using gradient ascent to generate adversarial sentences -- mutation();
        (2) Select the best samples which induce longest output sentences -- select_best();
        (3) Save the adversarial samples -- adv_his.
        """
        assert len(text) != 1
        # torch.autograd.set_detect_anomaly(True)
        ori_len, (best_adv_text, best_len), (current_adv_text, current_len) = self.prepare_attack(text)
        # adv_his = [(deepcopy(current_adv_text), deepcopy(current_len), 0.0)]
        adv_his = []
        modify_pos = []
        pbar = tqdm(range(self.max_per))
        t1 = time.time()

        for it in pbar:
            loss_list = self.compute_loss([current_adv_text])
            loss = sum(loss_list)
            self.model.zero_grad()
            loss.backward()
            grad = self.embedding.grad
            new_strings = self.mutation(current_adv_text, grad, modify_pos)

            if new_strings:
                current_adv_text, current_len = self.select_best(new_strings)
                log_str = "%d, %d, %.2f" % (it, len(new_strings), best_len / ori_len)
                pbar.set_description(log_str)

                if current_len > best_len:
                    best_adv_text = deepcopy(current_adv_text)
                    best_len = current_len
                t2 = time.time()
                adv_his.append((best_adv_text, int(best_len), t2 - t1))

        if adv_his:
            return True, adv_his
        else:
            return False, [(deepcopy(current_adv_text), deepcopy(current_len), 0.0)]

In [19]:
class WordAttacker(SlowAttacker):
    def __init__(self, 
                 device,
                 tokenizer,
                 model,
                 max_len=64,
                 max_per=3):
        super(WordAttacker, self).__init__(device, tokenizer, model, max_len, max_per)

    def compute_loss(self, text):
        scores, seqs, pred_len = self.compute_score(text) # [T X V], [T], [1]
        loss_list = self.leave_eos_target_loss(scores, seqs, pred_len)
        # loss_list = self.leave_eos_loss(scores, pred_len)
        return loss_list
    

    def token_replace_mutation(self, current_adv_text, grad, modified_pos):
        new_strings = []
        current_ids = self.tokenizer(current_adv_text, return_tensors="pt", padding=True).input_ids[0]
        base_ids = current_ids.clone()
        for pos in modified_pos:
            t = current_ids[0][pos]
            grad_t = grad[t]
            score = (self.embedding - self.embedding[t]).mm(grad_t.reshape([-1, 1])).reshape([-1])
            index = score.argsort()
            for tgt_t in index:
                if tgt_t not in self.specical_token:
                    base_ids[pos] = tgt_t
                    break

        for pos, t in enumerate(current_ids):
            if t not in self.specical_id:
                cnt, grad_t = 0, grad[t]
                score = (self.embedding - self.embedding[t]).mm(grad_t.reshape([-1, 1])).reshape([-1])
                index = score.argsort()
                for tgt_t in index:
                    if tgt_t not in self.specical_token:
                        new_base_ids = base_ids.clone()
                        new_base_ids[pos] = tgt_t
                        candidate_s = self.tokenizer.decode(new_base_ids, skip_special_tokens=True)
                        new_strings.append(candidate_s)
                        cnt += 1
                        if cnt >= 50:
                            break

        return new_strings


    def mutation(self, current_adv_text, grad, modify_pos):
        new_strings = self.token_replace_mutation(current_adv_text, grad, modify_pos)
        return new_strings

### Inference pipeline

In [20]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(preds, labels, metric, tokenizer):
    if not isinstance(preds, list):
        preds = [preds]
    if not isinstance(labels, list):
        labels = [labels]
    preds, labels = postprocess_text(preds, labels)
    result = metric.compute(predictions=preds, references=labels)
    return result['score']


def inference(sentence, label, model, tokenizer, metric, device):
    input_ids = tokenizer(sentence, return_tensors="pt").input_ids
    input_ids = input_ids.to(device)
    t1 = time.time()
    outputs = model.generate(input_ids, max_length=64, do_sample=False)
    output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    t2 = time.time()
    prediction_len = len(output.split())
    eval_score = compute_metrics(output, label, metric, tokenizer)

    print("")
    success, adv_his = attacker.run_attack(sentence)
    print("\nU--{}".format(sentence))
    print("G--{}".format(output))
    print("(length: {}, latency: {:.3f}, BLEU: {:.3f})".format(
        prediction_len, t2-t1, eval_score,
    ))

    if success:
        print("U'--{}".format(adv_his[-1][0]))
    else:
        print("Attack failed!")

    input_ids = tokenizer(adv_his[-1][0], return_tensors="pt").input_ids
    input_ids = input_ids.to(device)
    t1 = time.time()
    outputs = model.generate(input_ids, max_length=64, do_sample=False)
    output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    t2 = time.time()
    prediction_len = len(output.split())
    print("G'--{}".format(output))
    eval_score = compute_metrics(output, label, metric, tokenizer)
    print("(length: {}, latency: {:.3f}, BLEU: {:.3f})".format(
        prediction_len, t2-t1, eval_score,
    ))

### Demo

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
attacker = WordAttacker(
        device=device,
        tokenizer=tokenizer,
        model=model,
        max_len=64,
        max_per=1,
    )

metric = load_metric("sacrebleu")

In [23]:
# Demo 1
input_text = "Can't believe the kid grew up so quick."
output_text = "Yeah, kids grow up so quickly."
inference(input_text, output_text, model, tokenizer, metric, device)

# Demo 2
input_text = "How would I start rock climbing?"
output_text = "You can google it. But I suggest you to find a local climbing gym and take a class."
inference(input_text, output_text, model, tokenizer, metric, device)

# Demo 3
input_text = "How often do you use computers?"
output_text = "Almost every week. I use them for work and personal use."
inference(input_text, output_text, model, tokenizer, metric, device)

0, 500, 1.00: 100%|██████████| 1/1 [00:12<00:00, 12.86s/it]



U--Can't believe the kid grew up so quick.
G--I can't believe it. He's only a few months old.
(length: 10, latency: 0.189, BLEU: 3.386)
U'--Can Directions believe the kid grew up so quick.
G'--I can't believe it. My dad was an Army brat and he always said he wanted to be a pilot.
(length: 20, latency: 0.277, BLEU: 1.727)



0, 350, 1.00: 100%|██████████| 1/1 [00:11<00:00, 11.03s/it]



U--How would I start rock climbing?
G--If you know how to climb, you can google it.
(length: 10, latency: 0.211, BLEU: 12.046)
U'--How would I start rock waterfall?
G'--There are many ways to start a rock waterfall. There are many different types of water based on rock factors such as salinity, temperate, etc.
(length: 25, latency: 0.352, BLEU: 1.820)



0, 350, 1.00: 100%|██████████| 1/1 [00:09<00:00,  9.29s/it]



U--How often do you use computers?
G--Almost every week ends.
(length: 4, latency: 0.113, BLEU: 8.627)
U'--How sort do you use computers?
G'--I have a couple of computers. One is a gaming rig and the other is a storage unit.
(length: 18, latency: 0.235, BLEU: 2.708)


In [24]:
# Demo on BST test set
import random

def test_demo(device, model, tokenizer, attacker, max_num_samples=100, max_per=3):
    random.seed(2019)
    bst_dataset = load_dataset("blended_skill_talk")
    test_dataset = bst_dataset['test']
    ids = random.sample(range(len(test_dataset)), max_num_samples)

    sampled_test_dataset = test_dataset.select(ids)

    metric = load_metric("sacrebleu")
    ori_lens, adv_lens = [], []
    ori_bleus, adv_bleus = [], []
    ori_time, adv_time = [], []
    att_success = 0
    total_pairs = 0

    for i, instance in tqdm(enumerate(sampled_test_dataset)):
        if total_pairs >= max_num_samples:
            break

        for (sentence, label) in zip(instance['free_messages'], instance['guided_messages']):

            input_ids = tokenizer(sentence, return_tensors="pt").input_ids
            input_ids = input_ids.to(device)
            t1 = time.time()
            
            outputs = model.generate(input_ids, max_length=64, do_sample=False)
            output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
            t2 = time.time()
            pred_len = np.count_nonzero(outputs[0].cpu() != tokenizer.pad_token_id)
            eval_scores = compute_metrics(output, label, metric, tokenizer)
            
            ori_lens.append(pred_len)
            ori_bleus.append(eval_scores)
            ori_time.append(t2-t1)
            
            # Attack
            print("")
            success, adv_his = attacker.run_attack(sentence)
            print('\n')
            print("U--{}".format(sentence))
            print("G--{}".format(output))
            print("(length: {}, latency: {:.3f}, BLEU: {:.3f})".format(pred_len, t2-t1, eval_scores))

            if success:
                # print("Attack Succeed!")
                print("U'--{}".format(adv_his[-1][0]))
            else:
                print("Attack failed!")

            input_ids = tokenizer(adv_his[-1][0], return_tensors="pt").input_ids
            input_ids = input_ids.to(device)
            t1 = time.time()
            outputs = model.generate(input_ids, max_length=64, do_sample=False)
            output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
            t2 = time.time()
            adv_pred_len = np.count_nonzero(outputs[0].cpu() != tokenizer.pad_token_id)
            print("G'--{}".format(output))
            eval_scores = compute_metrics(output, label, metric, tokenizer)
            print("(length: {}, latency: {:.3f}, BLEU: {:.3f})".format(adv_pred_len, t2-t1, eval_scores))

            adv_lens.append(adv_pred_len)
            adv_bleus.append(eval_scores)
            adv_time.append(t2-t1)

            att_success += (adv_pred_len > pred_len)
            total_pairs += 1

            if total_pairs >= max_num_samples:
                break


    # Summarize eval results
    ori_len = np.mean(ori_lens)
    adv_len = np.mean(adv_lens)
    ori_bleu = np.mean(ori_bleus)
    adv_bleu = np.mean(adv_bleus)
    ori_t = np.mean(ori_time)
    adv_t = np.mean(adv_time)
    print("Original output length: {:.3f}, latency: {:.3f}, BLEU: {:.3f}".format(ori_len, ori_t, ori_bleu))
    print("Adversarial output length: {:.3f}, latency: {:.3f}, BLEU: {:.3f}".format(adv_len, adv_t, adv_bleu))
    print("Attack success rate: {:.2f}%".format(100*att_success/total_pairs))

In [25]:
max_num_samples = 5
max_per = 1
test_demo(device, model, tokenizer, attacker, max_num_samples, max_per)

  0%|          | 0/3 [00:00<?, ?it/s]

0it [00:00, ?it/s]


0, 550, 1.00: 100%|██████████| 1/1 [00:15<00:00, 15.28s/it]




U--I have two daughters. Do you have any children?
G--Yes, I have a son and daughter. How about you?
(length: 16, latency: 0.161, BLEU: 30.266)
U'--I have twoifix. Do you have any children?
G'--No, I don't. But I wish someday after getting married I will have my own. Do you like kids?
(length: 27, latency: 0.275, BLEU: 1.958)




0, 600, 1.00: 100%|██████████| 1/1 [00:14<00:00, 14.17s/it]


...



U--Wonderful. I am sure they will appreciate that! I like to photograph nature in my free time.
G--I love taking pictures of nature in my free time.
(length: 14, latency: 0.152, BLEU: 1.815)
U'--Wonderful. I am sure they will appreciate that! I took to photograph nature in my free time.
G'--It's nice to have a hobby, especially when you have free time. What kinds of things do you do?
(length: 26, latency: 0.282, BLEU: 2.443)




0, 1300, 1.00: 100%|██████████| 1/1 [00:45<00:00, 45.59s/it]




U--That's okay! Yes, I mostly photograph nature, but I like to photograph other things as well, like people and landmarks.
G--That's interesting. I like to photograph nature in general. People and landmarks are fascinating to me.
(length: 23, latency: 0.233, BLEU: 2.167)
U'--That's okay! Yes, I mostly photograph nature, but I like pip photograph other things as well, like people and landmarks.


1it [02:21, 141.66s/it]

G'--That's interesting. I like to photograph nature in general, but I also like being able to photograph other things, like people and landmarks.
(length: 31, latency: 0.335, BLEU: 1.650)
Original output length: 17.600, latency: 0.183, BLEU: 8.206
Adversarial output length: 28.400, latency: 0.301, BLEU: 2.022
Attack success rate: 100.00%
